## Instructions

### Fluidsynth
make sure that fluidsynth is set up and running as an ALSA daemon

In `/etc/conf.d/fluidsynth`:

    SOUND_FONT=/usr/share/soundfonts/FluidR3_GM2-2.sf2
    AUDIO_DRIVER=alsa
    OTHER_OPTS='-is -g 4 -m alsa_seq -r 48000'
    
and start the service:

    systemctl start fluidsynth.service
    
    
### mido
portmidi should be installed to correctly open up midi ports using mido

In [1]:
import os, time
import mido
import numpy as np

In [2]:
def fs_play(fname, vol=2):
    print('playing midi')
    os.system('fluidsynth -a alsa -m alsa_seq -g {} -l -i /usr/share/soundfonts/FluidR3_GM2-2.sf2 {}'.format(vol, fname))

In [3]:
fs_play('midi/schuim-1.mid', 2)

playing midi


In [12]:
def fs_port():
    ret = os.popen('aconnect -o').read()
    return ret

Open output

In [20]:
output = None
print(mido.get_output_names())
for o in mido.get_output_names():
    if 'Synth input' in o:
        output = mido.open_output(o)
print(output)

['Midi Through Port-0', 'Synth input port (30000:0)']
<open output 'Synth input port (30000:0)' (PortMidi/ALSA)>


#### Events, Notes, and Chords

In [5]:
class Event:
    ''' An event lasts for a duration of time and contians a note, multiple notes (chord), or nothing (rest)
    it also contains information about the number of occurences other events will happen after it '''
    # Note(s) of the event
    notes = []
    # Duration of the event
    dur = 1.0
    # Keeps track of the number of times the other events follow this event
    next_count = {}
    
    def __init__(self, notes, dur=1.0):
        self.notes = notes
        self.dur = dur
    
    def add_next(self, event):
        if event not in self.next_count:
            self.next_count[event] = 1
        else:
            self.next_count[event] += 1
            
    def choose_next(self):
        ''' Chooses the next event based on the observation of events having followed this event in the past '''
        s = sum(self.next_count.values())
        # Choose a next event
        i = 0
        sl = list(sorted(self.next_count.items(), key=lambda x: x[1], reverse=True))
        n = sl[i]
        while np.random.rand() > n[1]/s:
            i += 1
            if i > len(sl)-1:
                n = sl[0]
                break
            n = sl[i]
        return n[0]
    
    def hash(self):
        ''' Because events will be stored in a dict. Hashes by concatinating duration with notes '''
        if len(self.notes) == 0:
            return str(int(self.dur*1000))
        else:
            h = ''.join([ str(n.val) for n in
                            sorted(self.notes, key=lambda x: x.val) ])
            return h+str(int(self.dur*1000))
        #return sum([ i*127 for i in self.notes ])# + self.dur*127
    
    def __repr__(self):
        return '< Event {} >'.format(self.hash())
    
class Note:
    val = 0
    vel = 127
    
    def __init__(self, val, vel=127):
        self.val = int(val)
        self.vel = int(vel)
        self.next_count = {}
        
    def __repr__(self):
        return '< Note {} >'.format(self.val)
    

### Parse midi

In [21]:
events = {}
c_events = []
p_events = []
c_notes = []
p_notes = []
for msg in mido.MidiFile('midi/schub_d960_1.mid'):
    if not isinstance(msg, mido.MetaMessage):
        #print(msg)
        #output.send(msg)
        if msg.time == 0.0:
            if msg.type == 'note_on':
                c_notes.append(Note(msg.note, vel=msg.velocity))
        else:
            for n in c_notes:
                e = Event([n], msg.time)
                c_events.append(e)
                if e.hash() not in events:
                    events[e.hash()] = e
                    
                if len(p_events) > 0:
                    for pe in p_events:
                        pe.add_next(e)
                        
            if len(c_events) > 0:
                p_events = c_events
                c_events = []
            c_notes = []

In [22]:
print(len(events))
print(len(events['43121'].next_count))
for i in range(20):
    print(events['43121'].choose_next())

4325
62432
< Event 6925 >
< Event 6110 >
< Event 41128 >
< Event 58196 >
< Event 41128 >
< Event 41128 >
< Event 53203 >
< Event 41128 >
< Event 41128 >
< Event 53185 >
< Event 4925 >
< Event 41128 >
< Event 41128 >
< Event 486 >
< Event 554 >
< Event 41128 >
< Event 6124 >
< Event 66306 >
< Event 41128 >
< Event 413 >


### Make a song

In [ ]:
c_event = events['43121']
num_events = 0
while num_events < 100:
    num_events += 1
    print(c_event)
    for n in c_event.notes:
        # Turn on a note
        msg = mido.Message('note_on', note=n.val, velocity=64)
        output.send(msg)
    print(c_event)
    time.sleep(1.0)
    
    for n in c_event.notes:
        # Turn off a note
        msg = mido.Message('note_off', note=n.val, velocity=n.vel)
        output.send(msg)
        
    c_event = c_event.choose_next()
    

< Event 43121 >
< Event 43121 >
< Event 50196 >
< Event 50196 >
< Event 627 >
< Event 627 >
< Event 41128 >
< Event 41128 >
< Event 64225 >
< Event 64225 >
< Event 41128 >
< Event 41128 >
< Event 58125 >
< Event 58125 >
< Event 53369 >
< Event 53369 >
< Event 5211 >
< Event 5211 >
< Event 41128 >
< Event 41128 >
< Event 41128 >
< Event 41128 >
< Event 7430 >
< Event 7430 >
< Event 75163 >
< Event 75163 >
< Event 57131 >
< Event 57131 >
< Event 41128 >
< Event 41128 >
< Event 2928 >
< Event 2928 >
< Event 8042 >
< Event 8042 >
< Event 41128 >
< Event 41128 >
< Event 41128 >
< Event 41128 >
< Event 631050 >
< Event 631050 >
< Event 41128 >
< Event 41128 >
< Event 9128 >
< Event 9128 >
< Event 41128 >